In [12]:
import os
import sys

import openai
from dotenv import load_dotenv, find_dotenv

sys.path.append('../..')

# read local .env file
_ = load_dotenv(find_dotenv())

openai.api_key = os.environ['OPENAI_API_KEY']
openai.api_base = os.environ['OPENAI_API_BASE']

# Prepare

Download SQLite: https://www.sqlite.org/download.html

sqlite3 ./files/Chinook.db
```
> .read ./files/Chinook_Sqlite.sql
> SELECT * FROM Artist LIMIT 10;
```

## SQLDatabaseToolkit

In [13]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.sql_database import SQLDatabase

In [14]:
db = SQLDatabase.from_uri("sqlite:///files/Chinook.db")
toolkit = SQLDatabaseToolkit(db=db, llm=ChatOpenAI(temperature=0, model_name="gpt-4"))

In [15]:
agent_executor = create_sql_agent(
    llm=ChatOpenAI(temperature=0, model_name="gpt-4"),
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

In [16]:
agent_executor.run("Describe the playlisttrack table")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Thought:I now know that there is a table called `PlaylistTrack` in the database. I should query its schema to understand its structure and describe it.
Action: sql_db_schema
Action Input: "PlaylistTrack"
Observation: 
CREATE TABLE "PlaylistTrack" (
	"PlaylistId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	PRIMARY KEY ("PlaylistId", "TrackId"), 
	FOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), 
	FOREIGN KEY("PlaylistId") REFERENCES "Playlist" ("PlaylistId")
)

/*
3 rows from PlaylistTrack table:
PlaylistId	TrackId
1	3402
1	3389
1	3390
*/
Thought:I now know that the `PlaylistTrack` table has two columns, `PlaylistId` and `TrackId`, which are both integers and form a primary key. There are also foreign keys referencing the `Playlist` and `Track` tables. I can describe this tab

'The `PlaylistTrack` table contains data about which tracks are included in each playlist, with two columns: `PlaylistId` (an integer that references a specific playlist in the `Playlist` table) and `TrackId` (an integer that references a specific track in the `Track` table). Each unique pair of `PlaylistId` and `TrackId` forms a primary key, ensuring that each track can only appear once in any given playlist.'

In [17]:
agent_executor.run("Describe the playlistsong table")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Thought:The table that seems most relevant to the question is "PlaylistTrack". I will now query its schema.
Action: sql_db_schema
Action Input: PlaylistTrack
Observation: 
CREATE TABLE "PlaylistTrack" (
	"PlaylistId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	PRIMARY KEY ("PlaylistId", "TrackId"), 
	FOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), 
	FOREIGN KEY("PlaylistId") REFERENCES "Playlist" ("PlaylistId")
)

/*
3 rows from PlaylistTrack table:
PlaylistId	TrackId
1	3402
1	3389
1	3390
*/
Thought:The `PlaylistTrack` table has two columns: `PlaylistId` and `TrackId`. Both of these columns are integers and cannot be null. The primary key is a combination of both columns, ensuring that each track can only appear once in a playlist. There are also foreign keys referencing th

'The `PlaylistTrack` table contains two integer columns, `PlaylistId` and `TrackId`, which together form a primary key. These foreign keys reference the `Playlist` and `Track` tables, respectively, making this table a many-to-many relationship between playlists and tracks.'

In [18]:
agent_executor.run(
    "List the total sales per country. Which country's customers spent the most?"
)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Thought:The `Invoice` table seems relevant as it might contain sales information. I will query its schema to see the available columns.
Action: sql_db_schema
Action Input: "Invoice"
Observation: 
CREATE TABLE "Invoice" (
	"InvoiceId" INTEGER NOT NULL, 
	"CustomerId" INTEGER NOT NULL, 
	"InvoiceDate" DATETIME NOT NULL, 
	"BillingAddress" NVARCHAR(70), 
	"BillingCity" NVARCHAR(40), 
	"BillingState" NVARCHAR(40), 
	"BillingCountry" NVARCHAR(40), 
	"BillingPostalCode" NVARCHAR(10), 
	"Total" NUMERIC(10, 2) NOT NULL, 
	PRIMARY KEY ("InvoiceId"), 
	FOREIGN KEY("CustomerId") REFERENCES "Customer" ("CustomerId")
)

/*
3 rows from Invoice table:
InvoiceId	CustomerId	InvoiceDate	BillingAddress	BillingCity	BillingState	BillingCountry	BillingPostalCode	Total
1	2	2009-01-01 00:00:00	Theodor-Heuss-St

'Customers from the USA spent the most with a total of $523.06, followed by customers from France who spent $195.09 and customers from Germany who spent $156.48.'

In [19]:
agent_executor.run(
    "Show the total number of tracks in each playlist. The Playlist name should be included in the result."
)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Thought:The `Playlist` and `PlaylistTrack` tables seem relevant. I should query their schema to see what columns they have.
Action: sql_db_schema
Action Input: Playlist, PlaylistTrack
Observation: 
CREATE TABLE "Playlist" (
	"PlaylistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("PlaylistId")
)

/*
3 rows from Playlist table:
PlaylistId	Name
1	Music
2	Movies
3	TV Shows
*/


CREATE TABLE "PlaylistTrack" (
	"PlaylistId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	PRIMARY KEY ("PlaylistId", "TrackId"), 
	FOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), 
	FOREIGN KEY("PlaylistId") REFERENCES "Playlist" ("PlaylistId")
)

/*
3 rows from PlaylistTrack table:
PlaylistId	TrackId
1	3402
1	3389
1	3390
*/
Thought:I can construct a query to join the `Playlist` and `Playlis

'The query above will provide the total number of tracks in each playlist, with the playlist name included in the result.'

In [20]:
agent_executor.run("Who are the top 3 best selling artists?")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Thought:The 'Artist' and 'Invoice' tables seem relevant. I should query their schema to see what columns they have.
Action: sql_db_schema
Action Input: Artist, Invoice
Observation: 
CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/


CREATE TABLE "Invoice" (
	"InvoiceId" INTEGER NOT NULL, 
	"CustomerId" INTEGER NOT NULL, 
	"InvoiceDate" DATETIME NOT NULL, 
	"BillingAddress" NVARCHAR(70), 
	"BillingCity" NVARCHAR(40), 
	"BillingState" NVARCHAR(40), 
	"BillingCountry" NVARCHAR(40), 
	"BillingPostalCode" NVARCHAR(10), 
	"Total" NUMERIC(10, 2) NOT NULL, 
	PRIMARY KEY ("InvoiceId"), 
	FOREIGN KEY("CustomerId") REFERENCES "Customer" ("CustomerId")
)

/*
3 rows f

'Here are the top 3 best-selling tracks:'